In [9]:
# Import des librairies
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Import des données
clicks_sample = pd.read_csv('../1-Données/clicks_sample.csv', sep=",")
clicks_folder = "../1-Données/clicks/"
articles_embeddings = pd.read_pickle('../1-Données/articles_embeddings.pickle')

In [11]:
# Dataframe de tous les clics
df_clicks = pd.DataFrame()
for f in os.listdir(clicks_folder):
    df = pd.read_csv(clicks_folder + f)
    df_clicks = pd.concat([df_clicks, df])

In [21]:
# Enregistrement du dataframe global dans un fichier pickle
df_clicks.to_pickle(clicks.pickle)

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2564,10051,1508211372158328,1508211372000,2,84911,1508211557302,4,3,2,1,25,1
2565,322896,1508211376302329,1508211376000,2,30760,1508211672520,4,1,17,1,25,2
2566,322896,1508211376302329,1508211376000,2,157507,1508211702520,4,1,17,1,25,2
2567,123718,1508211379189330,1508211379000,2,234481,1508211513583,4,3,2,1,25,2


In [14]:
# Entrée
id_user = 1

In [15]:
# Fonction
# Sort un dataframe comprenant les 5 articles les plus proches de l'article ainsi que leur score
def get_best_articles(id_article):
    similarities = np.squeeze(cosine_similarity(articles_embeddings[:,:], articles_embeddings[id_article,:].reshape(1, 250)))
    indices_max = np.argsort(-similarities)[1:6]
    return pd.DataFrame([indices_max, similarities[indices_max]], index=['article', 'score'])

In [16]:
# Liste des articles vus par un utilisateur
user_articles = df_clicks.loc[df_clicks['user_id']==id_user]['click_article_id']

# Création du dataframe contenant les recommandations
df_best_articles = pd.DataFrame()

# Parcours des articles vus par l'utilisateur
for i, article in user_articles.iteritems():
    # Récupère 5 plus proches articles et les concatène dans un df
    df = get_best_articles(article)
    df_best_articles = pd.concat([df_best_articles, df], axis=1)

# Tri de tous les articles de l'utilsateur selon leur score
df_best_articles.sort_values(by=['score'], axis=1, ascending=False)
df_best_articles.columns = range(len(df_best_articles.columns))

# Liste des 5 meilleures recommandations 
results = []
i = 0
j = 0
while j < 5:
    if  not df_best_articles[i]["article"] in user_articles.unique():
        results.append(int(df_best_articles[i]["article"]))
        j = j + 1
    i = i + 1

In [20]:
# Sortie
results

[60509, 235312, 235410, 108443, 234819]